##Prepare meteorological data for WOFOST run on gridded data

This notebook reads MeteoSwiss climate data [Tmin, Tmax](https://www.meteoswiss.admin.ch/dam/jcr:818a4d17-cb0c-4e8b-92c6-1a1bdf5348b7/ProdDoc_TabsD.pdf) and agricultural exposure data (or the spatial_units.gpkg file) as described in [Portmann et al. 2023](https://egusphere.copernicus.org/preprints/2023/egusphere-2023-2598/) and produces one .csv file for each meteorological variable with time as rows and grid cell IDs (of all 1x1km grid cells with exposure) as columns.

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import os

First, define directories of gridded exposure data (polygons), the MeteoSwiss data and the directory to store the output data

In [2]:
#produce MCH Data (comment if already done so)

#INPUT DATA
datadir_MCH='O:/Data-Raw/27_Natural_Resources-RE/99_Meteo_Public/MeteoSwiss_netCDF/__griddedData/lv95/'
datadir_exposure='C:/Users/F80840370/projects/scClim/climada/data/scClim/exposure/GIS/Weizen_Mais_Raps_Gerste_polygons.gpkg'

#OUTPUT DATA
OUTDIR='C:/Users/F80840370/projects/scClim/wofost/winter_wheat_phenology/meteo_data/MeteoSwiss/'

startyear=1971
endyear=2021
variables=['TminD','TmaxD']
filename_base='TminTmax_daily_{}_{}'.format(startyear,endyear)

Create spatial units file from exposure or just read it if it is there

In [3]:
#create from exposure
units=gpd.read_file(datadir_exposure)
units=units[units.n_fields>0]
units_lv95=units.to_crs(crs=2056)
units_lv95['X']=np.round(units_lv95.centroid.x)
units_lv95['Y']=np.round(units_lv95.centroid.y)
units_lv95['ID']=units_lv95.index
#save
units_lv95.to_file("spatial_units.gpkg", driver="GPKG")
#read
#units_lv95.from_file("spatial_units.gpkg", driver="GPKG")

Read gridded MeteoSwiss Climate Data from Agroscope Server. Make sure to use .load() to load the data into memory. Otherwise subsequent computations may take a long time. This step takes quite some time (up to 1h is to be expected)

In [41]:
#get list of datafiles #1961-2022
mch_data={}
for var in variables:
    datafiles=[datadir_MCH+str(var)+'_ch01r.swiss.lv95_'+str(year)+'01010000_'+str(year)+'12310000.nc' for year in range(startyear,endyear+1)]
    #read datafiles
    print('reading data for {} from {}...'.format(var,datadir_MCH))
    mch_data[var]=xr.open_mfdataset(datafiles,concat_dim = 'time',combine='nested',coords = 'minimal')
    #to be able to later run this quickly we need to load the dataset
    mch_data[var].load()

147475    2484500.0
147476    2484500.0
147477    2484500.0
148112    2485500.0
148116    2485500.0
            ...    
351650    2803500.0
351651    2803500.0
354204    2807500.0
363869    2822500.0
365790    2825500.0
Name: X, Length: 12742, dtype: float64

For each variable read MeteoSwiss data and create for each variable a DataFrame with time as column and spatial units (ID) as rows. Write one file per variable for all years

In [30]:
for var in variables:
      datalist=[pd.Dataframe({'time': mch_data[var].time.values})]
      for ind in units_lv95.index:
            X=units_lv95.iloc[ind].X
            Y=units_lv95.iloc[ind].Y
            ID=units_lv95.iloc[ind].ID
            data=mch_data[var].sel(E=X,N=Y,method='nearest')
            datalist.append=pd.Dataframe({ID: data[var].values})
            
      df_out=pd.concat(datalist,axis=1)
      filename='{}_{}_{}'.format(var,startyear,endyear)
      df_out.to_csv(OUTDIR+filename+'.csv')


147475    147475
147476    147476
147477    147477
148112    148112
148116    148116
           ...  
351650    351650
351651    351651
354204    354204
363869    363869
365790    365790
Name: ID, Length: 12742, dtype: int64

Voilá, you are ready to run wofost

In [10]:
with open('test.txt') as src:
    bla=src.read()

In [12]:
bla.split()

['1234']